In [1]:
# Made by: João Vitor Andrioli de Souza

## Internal Functions

In [2]:
class CRFinternalFunctions:
    def sent2features(self,sent,TokensEmVolta,atributos,fitting = False):        
        if fitting == True:
            return [self.word2features(list(zip(*list(zip(*sent))[:-1])), i,TokensEmVolta,atributos) for i in range(len(sent))]
        else:
            st = []
            for sn in sent:
                if type(sn) == str:
                    st.append((sn,))
                else:
                    st.append(sn)
            return [self.word2features(st, i,TokensEmVolta,atributos) for i in range(len(sent))]
    
    def sent2labels(self,sent):
        return [tokenlabeladc[-1] for tokenlabeladc in sent]
    
    
    def word2features(self,doc, i,TokensEmVolta,atributosOriginal): #TokensEmVolta    deve ser     1,2,3 ou 4
   
        def vogaisconsecutivas(word):
            vogal =  [1 if letra in 'aeiou' else 0 for letra in word]
            for y in vogal:
                vogal = [vogal[x-1] + 1 if vogal[x-1] > 0 and vogal[x] > 0 else vogal[x] for x in range(len(vogal))] 
            if len(vogal) == 0:
                return '0'
            return str(max(number for number in vogal))

        def consoantesconsecutivas(word):
            vogal =  [1 if letra in 'bcdfghjklmnpqrstvwxyz' else 0 for letra in word]
            for y in vogal:
                vogal = [vogal[x-1] + 1 if vogal[x-1] > 0 and vogal[x] > 0 else vogal[x] for x in range(len(vogal))]    
            if len(vogal) == 0:
                return '0'
            return str(max(number for number in vogal))

        def MaiusculoEmVolta(doc,i,emvolta):
            try:
                return False if False in [False if upmvolt(doc[i-x][0]) == False else True for x in range(-emvolta,emvolta+1)] else True
            except:
                return MaiusculoEmVolta(doc,i,emvolta-1)


        upmvolt =          lambda word: word.isupper() or word.isnumeric()
        contavogais =      lambda word: str(len(list(filter(lambda letras: letras in 'aeiou',word)))) #Abbreviation nao tem acento
        semvogal =         lambda word: True if contavogais(word) == 0 else False
        maior2consoantes = lambda word: True if int(consoantesconsecutivas(word)) > 2 else False
        possuiacento =     lambda word: True if True in [True if x in 'áéíóúãõà' else False for x in word] else False
        hasPunctuation =   lambda word: True if True in [True if x in ".,:\\|^~'*-+!=&%#@" else False for x in word] else False
        lenghtUm =         lambda word: True if len(word) == 1 else False
        hasHyphen =        lambda word: True if True in [True if x == "-" else False for x in word] else False
        letterAndDigit =   lambda word: True if True in [True if x in "qwertyuiopasdfghjklçzxcvbnm" and True in [True if y in "0987654321" else False for y in word] else False for x in word] else False
        hasUpper =         lambda word: True if True in [True if x in "QWERTYUIOPÇLKJHGFDSAZXCVBNM" else False for x in word] else False
        hasUnderscore =    lambda word: True if True in [True if x == "_" else False for x in word] else False
        hasBrackets =      lambda word: True if True in [True if x in "][}{()" else False for x in word] else False
        DigitoVirgulaDigito = lambda word: True if True in [True if  word[x].isdigit() and (word[x+1] == "." or word[x+1] == ",") and word[x+2].isdigit() else False for x in range(len(word)-2)] else False


        def generate_ngrams(snt, n, posicao):

            tokens = []
            for w in snt:
                try:
                    tokens.append(w[0].lower())
                except:
                    tokens.append("")
                    
            ngrams = zip(*[tokens[i:] for i in range(n)])
            res = [" ".join(ngram) for ngram in ngrams]

            try:
                return res[posicao]
            except:
                try:
                    return res[len(res)-1]
                except:
                    return '-1'

        #################################
        features = []
        atributos = atributosOriginal
        
        for posicao in range(-TokensEmVolta,TokensEmVolta+1):
            eos = False 
            bos = False        
            if i-posicao == 0:
                bos = True        # Indicate that it is the 'beginning of a document'
            if i-posicao == len(doc):
                eos = True # Indicate that it is the 'end of a document'
                
            if i-posicao >= 0 and i-posicao < len(doc):
             
            
                try:
                    word = doc[i-posicao][0]
                except:
                    break
             
            
            
                featuresAdicionais = []
                if len(doc[i-posicao]) > 1:
                    for FAD in doc[i-posicao][1:]:
                        featuresAdicionais.append(FAD)
                
                posicaostr = str(-posicao)
                
                lista_atributos = [
                    posicaostr+':word.normal=' + word,
                    posicaostr+':word.isupper=%s' % word.isupper(),
                    posicaostr+':word.istitle=%s' % word.istitle(),
                    posicaostr+':word.isdigit=%s' % word.isdigit(),
                    posicaostr+':word.size=%s' % len(word),
                    posicaostr+':numerovogais=' + contavogais(word),
                    posicaostr+':vogaisconsecutivas='+vogaisconsecutivas(word),
                    posicaostr+':consoantesconsecutivas='+consoantesconsecutivas(word),
                    posicaostr+':maior2consoantes=%s'% maior2consoantes(word),    
                    posicaostr+':possuiacento=%s'% possuiacento(word),   
                    posicaostr+':semvogal=%s'% semvogal(word),
                    posicaostr+':sentence.isupper=%s' % MaiusculoEmVolta(doc,i,TokensEmVolta),
                    posicaostr+':word.tempontuacao=%s' % hasPunctuation(word),
                    posicaostr+':word.islower=%s' % word.islower(),
                    posicaostr+':word.tamanhoum=%s' % lenghtUm(word),
                    posicaostr+':word.letraenumero=%s' % letterAndDigit(word),
                    posicaostr+':word.temmaiuscula=%s' % hasUpper(word),
                    posicaostr+':word.temunderscore=%s' % hasUnderscore(word),
                    posicaostr+':word.tembrackets=%s' % hasBrackets(word),
                    posicaostr+':word.temhyphen=%s' % hasHyphen(word),
                    posicaostr+':word.numeroflutuante=%s' % DigitoVirgulaDigito(word),
                    posicaostr+':word.bigram=' + generate_ngrams(doc, 2, i-posicao),
                    posicaostr+':word.trigram=' + generate_ngrams(doc, 3, i-posicao)
                ]
    
                for indexa,FAD in enumerate(featuresAdicionais):
                     lista_atributos.append(posicaostr+':featureAdicional='+ str(indexa) + str(FAD))
    
            
                atributos_escolhidos = [posicaostr+':word.lower=' + word.lower(),'EOS=%s'%eos,'BOS=%s'%bos]
                
                if len(atributos) != len(lista_atributos):
                    #print("mascara de atributos selecionados possui tamanho errado")
                    #print("tamanho atributos selecionados:",len(atributos))
                    #print("tamanho atributos usados      :",len(lista_atributos))
                    #print("treinando com todos os atributos!")
                    atributos = []
                    
                if len(atributos) == 0:
                    atributos = [1 for _ in range(len(lista_atributos))]

                    
                for l in range(len(atributos)):
                    if atributos[l] == 1:
                        atributos_escolhidos.append(lista_atributos[l])
                
                atributos = atributosOriginal
                
                features.extend(atributos_escolhidos)
                
    
        return features
    

## User Classes

In [3]:
class CRFmodel:
    def __init__(self,*sklearnCRFmodel):
        self.cro = []
        if len(sklearnCRFmodel) > 0:
            self.sklearnCRFmodel = sklearnCRFmodel[0]
        else:
            import sklearn_crfsuite
            self.sklearnCRFmodel = sklearn_crfsuite.CRF( algorithm='pa', c = 1.0 ,pa_type = 1 , max_iterations=100, all_possible_transitions=True)
        self.tokensAround = 4
        
        
    def fit(self,dados):
        X = [CRFinternalFunctions().sent2features(s,self.tokensAround,self.cro,fitting = True) for s in dados]
        y = [CRFinternalFunctions().sent2labels(s) for s in dados]
        
        self.sklearnCRFmodel = self.sklearnCRFmodel.fit(X, y)
        
    def predict(self,Xraw):        
        X = [CRFinternalFunctions().sent2features(s,self.tokensAround,self.cro) for s in Xraw]

        return self.sklearnCRFmodel.predict(X)
    
    def selectFeatures(self,mask):
        self.cro = mask
    
class CRFutil:
    def saveModel(self,CRFmodel,modelNameFile):
        import pickle
        file = open(modelNameFile+".txt", "wb")
        pickle.dump(CRFmodel, file)

    def loadModel(self,modelNameFile):
        import pickle
        file = open(modelNameFile+".txt", "rb")
        return pickle.load(file)
    
    def openConllFile(self,inputNameFile):
        with open(inputNameFile, 'r') as f:  
            sentenceS = []
            sentence  = []
            for line in f:
                line = line[:-1]
                if line.strip() == "":
                    sentenceS.append(sentence)
                    sentence  = []
                else:
                    sentence.append(line.split(" "))
        return sentenceS

## Input format

In [4]:
#corpus deve ser uma lista de sentencas, sentencas deve ser uma lista de tokens e cada token uma tripla(palavra,featureAdicional1,featureAdicional2,TAG)
#primeiro item é palavra, ultimo é label, os n do meio são features adicionais
samplecorpusformat = [[['POI', 'B-Abbreviation'],  ['DE', 'O'],  ['LAVAGEM', 'O'],  ['+', 'O'],  ['CURETA', 'O'],  ['DE', 'O'],  ['TECIDO', 'O'],  ['NECROTICO', 'O'],  ['.', 'O']], [['18', 'O'],  [':', 'O'],  ['00', 'O'],  [':', 'O'],  ['PACIENTE', 'O'],  ['RETORNOU', 'O'],  ['DO', 'O'],  ['CC', 'B-Abbreviation'],  ['LUCIDO', 'O'], [',', 'O'],  ['ORIENTADO', 'O'],  [',', 'O'],  ['COMUNICATIVO', 'O'],  [';', 'O'], ['MANTEM', 'O'],  ['AVP', 'B-Abbreviation'],  ['COM', 'O'],  ['STP', 'B-Abbreviation'], ['.', 'O']]]

#################################################

# You can use the openConllFile function if the data is a file in connl format

# create the CRF util class
utils = CRFutil()
inputNameFile = "AbbreviationDadostrain.txt"
TRAININGcorpus = utils.openConllFile(inputNameFile)

print("Number of sentences (TRAIN): ",len(TRAININGcorpus))
print(TRAININGcorpus[:2])

#################################################

inputNameFile = "AbbreviationDadosdev.txt"
PREDICTIONcorpus = utils.openConllFile(inputNameFile)

print("Number of sentences (TEST): ",len(PREDICTIONcorpus))
print(PREDICTIONcorpus[:2])


# The prediction data must not have the labels
def getSentencesAndFeaturesOnly(dados):
    newDados = []
    for sent in dados:
        try:
            newDados.append(list(zip(*list(zip(*sent))[:-1])))
        except:
            pass
    return newDados

PREDICTIONSentences = getSentencesAndFeaturesOnly(PREDICTIONcorpus)
print("Prediction format:")
print(PREDICTIONSentences[:2])


Number of sentences (TRAIN):  5740
[[['POI', 'B-Abbreviation'], ['DE', 'O'], ['LAVAGEM', 'O'], ['+', 'O'], ['CURETA', 'O'], ['DE', 'O'], ['TECIDO', 'O'], ['NECROTICO', 'O'], ['.', 'O']], [['18', 'O'], [':', 'O'], ['00', 'O'], [':', 'O'], ['PACIENTE', 'O'], ['RETORNOU', 'O'], ['DO', 'O'], ['CC', 'B-Abbreviation'], ['LUCIDO', 'O'], [',', 'O'], ['ORIENTADO', 'O'], [',', 'O'], ['COMUNICATIVO', 'O'], [';', 'O'], ['MANTEM', 'O'], ['AVP', 'B-Abbreviation'], ['COM', 'O'], ['STP', 'B-Abbreviation'], ['.', 'O']]]
Number of sentences (TEST):  1913
[[['ACESSO', 'O'], ['VENOSO', 'O'], ['PERIFERICO', 'O'], ['COM', 'O'], ['FLUIDOTERAPIA', 'O'], ['EM', 'O'], ['CURSO', 'O'], ['EM', 'O'], ['MSE', 'B-Abbreviation'], ['.', 'O']], [['ABDOME', 'O'], ['PLANO', 'O'], [',', 'O'], ['FLACIDO', 'O'], ['.', 'O']]]
Prediction format:
[['ACESSO', 'VENOSO', 'PERIFERICO', 'COM', 'FLUIDOTERAPIA', 'EM', 'CURSO', 'EM', 'MSE', '.'], ['ABDOME', 'PLANO', ',', 'FLACIDO', '.']]


## Using the CRF Model

In [5]:
# creating the CRF Model
modeloCRF = CRFmodel()

# training the model
modeloCRF.fit(TRAININGcorpus)

#################################################

# file name for saving and loading the model
modelNameFile = 'CRFtestmodel'

# create the CRF util class
utils = CRFutil()

# SAVING the model
utils.saveModel(modeloCRF,modelNameFile)

# LOADING the model
modeloCRF = utils.loadModel(modelNameFile)

#################################################

# predicting
# The prediction data must not have the labels
y_pred = modeloCRF.predict(PREDICTIONSentences)

print("Prediction:")
print(y_pred[:2])

# if you want to save the y_pred, you can use the same functions .saveModel and .loadModel, 
# because they are the basic pickle dump and load!

Prediction:
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Abbreviation', 'O'], ['O', 'O', 'O', 'O', 'O']]


## Optional Parameters

In [6]:
#You can define your owm crf model from sklearn
import sklearn_crfsuite
sklearnCRFmodel = sklearn_crfsuite.CRF( algorithm='pa', c = 1.0 ,pa_type = 1 , max_iterations=100, all_possible_transitions=True)
modeloCRF = CRFmodel(sklearnCRFmodel) 

In [7]:
#You can select only the features you want to use with a binary mask
featuresMask = dict()
featuresMask['word.normal']            = 1
featuresMask['word.isupper']           = 0
featuresMask['word.istitle']           = 0
featuresMask['word.isdigit']           = 0
featuresMask['word.size']              = 0
featuresMask['numerovogais']           = 1
featuresMask['vogaisconsecutivas']     = 1
featuresMask['consoantesconsecutivas'] = 0
featuresMask['maior2consoantes'  ]     = 1
featuresMask['possuiacento' ]          = 0
featuresMask['semvogal']               = 0
featuresMask['sentence.isupper']       = 0
featuresMask['word.tempontuacao']      = 1
featuresMask['word.islower']           = 1
featuresMask['word.tamanhoum']         = 0
featuresMask['word.letraenumero']      = 0
featuresMask['word.temmaiuscula']      = 1
featuresMask['word.temunderscore']     = 0
featuresMask['word.tembrackets']       = 1
featuresMask['word.temhyphen']         = 0
featuresMask['word.numeroflutuante']   = 0
featuresMask['word.bigram']            = 1
featuresMask['word.trigram']           = 0

#The input must be a binary list!
featuresSelected = list(featuresMask.values())
#featuresSelected = [1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0 ,1 ,0]

#You have to run this before calling .fit and .predict
modeloCRF.selectFeatures(featuresSelected)